In [ ]:
# !pip install pandas==1.0.1
import pandas as pd
df_ovitrap = pd.read_csv('/content/drive/My Drive/Group 4/WMP_data_2.csv')

In [ ]:
#convert collected_at to month only
df_ovitrap['collected_at'] = pd.to_datetime(df_ovitrap.collected_at)
df_ovitrap['col_month'] = df_ovitrap['collected_at'].dt.strftime('%B')
df_ovitrap

,trap_id,type,successful,collected_at,reporting_area,total_aegypti_caught,screening_total_aeg,screening_wmel_aeg,latitude,longitude,col_month
0,ASP-3U23MKVR,Aspirator,False,2020-11-26 10:48:11,Comuna 1,NaN,NaN,NaN,3.458973,-76.551185,November
1,ASP-3WQ6DML5,Aspirator,True,2020-11-26 10:48:55,Comuna 1,2.0,2.0,2.0,3.458807,-76.557513,November
2,ASP-33XRR0BC,Aspirator,False,2020-11-26 10:49:38,Comuna 1,NaN,NaN,NaN,3.457054,-76.561500,November
3,ASP-QUEZ5LU5,Aspirator,True,2020-11-26 10:50:21,Comuna 1,7.0,7.0,5.0,3.454069,-76.562949,November
4,ASP-3YE2DS4H,Aspirator,True,2020-11-26 10:50:53,Comuna 1,0.0,NaN,NaN,3.452289,-76.570642,November
...,...,...,...,...,...,...,...,...,...,...,...
1107,ASP-N4M1EF1Z,Aspirator,True,2021-04-14 12:20:32,Comuna 20,2.0,2.0,1.0,3.422684,-76.564239,April
1108,ASP-BE4OF193,Aspirator,True,2021-04-14 12:20:53,Comuna 20,24.0,10.0,3.0,3.422226,-76.560137,April
1109,ASP-J00IVPD3,Aspirator,True,2021-04-14 12:22:02,Comuna 20,13.0,10.0,2.0,3.423620,-76.558258,April
1110,ASP-XUW2X4F7,Aspirator,True,2021-04-14 12:22:19,Comuna 20,2.0,2.0,1.0,3.423295,-76.557037,April


In [ ]:
#put comunas into cats

categories = df_ovitrap['reporting_area'].unique()
# create a vector with the category names,
lencat = len(categories) # store the length of the categories vector to use later
# one hot encoding, create three new columns which are true for that particular species, and false if not
for a in categories: # loop over all the labels in categories
    # + concatenates two strings
      df_ovitrap[a] = pd.Series(df_ovitrap['reporting_area']==a).astype(int)

In [ ]:
#and months

categories = df_ovitrap['col_month'].unique()
# create a vector with the category names,
lencat = len(categories) # store the length of the categories vector to use later
# one hot encoding, create three new columns which are true for that particular species, and false if not
for a in categories: # loop over all the labels in categories
    # + concatenates two strings
      df_ovitrap[a] = pd.Series(df_ovitrap['col_month']==a).astype(int)

In [ ]:
df_ovitrap = df_ovitrap.drop(['reporting_area','col_month', 'collected_at'],axis=1)

In [ ]:
df_ovitrap = df_ovitrap.dropna(subset=['total_aegypti_caught'])
df_ovitrap = df_ovitrap[df_ovitrap.total_aegypti_caught != 0]
df_train = df_ovitrap.dropna(subset=['screening_total_aeg'])
df_test = df_ovitrap[df_ovitrap['screening_total_aeg'].isnull()]

In [ ]:
df_ovitrap

,trap_id,type,successful,total_aegypti_caught,screening_total_aeg,screening_wmel_aeg,latitude,longitude,Comuna 1,Comuna 18,Comuna 20,November,December,January,February,March,April,July
1,ASP-3WQ6DML5,Aspirator,True,2.0,2.0,2.0,3.458807,-76.557513,1,0,0,1,0,0,0,0,0,0
3,ASP-QUEZ5LU5,Aspirator,True,7.0,7.0,5.0,3.454069,-76.562949,1,0,0,1,0,0,0,0,0,0
10,ASP-IREL246U,Aspirator,True,17.0,17.0,6.0,3.457746,-76.583165,1,0,0,1,0,0,0,0,0,0
12,ASP-O4K94YY6,Aspirator,True,2.0,2.0,1.0,3.455246,-76.578502,1,0,0,0,1,0,0,0,0,0
13,ASP-IREL246U,Aspirator,True,15.0,15.0,4.0,3.457746,-76.583165,1,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1107,ASP-N4M1EF1Z,Aspirator,True,2.0,2.0,1.0,3.422684,-76.564239,0,0,1,0,0,0,0,0,1,0
1108,ASP-BE4OF193,Aspirator,True,24.0,10.0,3.0,3.422226,-76.560137,0,0,1,0,0,0,0,0,1,0
1109,ASP-J00IVPD3,Aspirator,True,13.0,10.0,2.0,3.423620,-76.558258,0,0,1,0,0,0,0,0,1,0
1110,ASP-XUW2X4F7,Aspirator,True,2.0,2.0,1.0,3.423295,-76.557037,0,0,1,0,0,0,0,0,1,0


In [ ]:
!pip install datawig

import datawig

#Initialize a SimpleImputer model
imputer = datawig.SimpleImputer(
    input_columns=['total_aegypti_caught','Comuna 1','Comuna 18','Comuna 20','November','December','January','February','March','April','July'], # column(s) containing information about the column we want to impute
    output_column= 'screening_total_aeg', # the column we'd like to impute values for
    output_path = 'imputer_model' # stores model data and metrics
    )

#Fit an imputer model on the train data
imputer.fit(train_df=df_train, num_epochs=50)

#Impute missing values and return original dataframe with predictions
imputed = imputer.predict(df_test)

2021-10-03 03:48:44,574 [INFO]  
========== start: fit model
2021-10-03 03:48:44,576 [WARNING]  Already bound, ignoring bind()
2021-10-03 03:48:44,614 [INFO]  Epoch[0] Batch [0-23]	Speed: 12806.40 samples/sec	cross-entropy=9.340570	screening_total_aeg-accuracy=0.000000
2021-10-03 03:48:44,647 [INFO]  Epoch[0] Train-cross-entropy=9.149827
2021-10-03 03:48:44,651 [INFO]  Epoch[0] Train-screening_total_aeg-accuracy=0.000000
2021-10-03 03:48:44,657 [INFO]  Epoch[0] Time cost=0.078
2021-10-03 03:48:44,662 [INFO]  Saved checkpoint to "imputer_model/model-0000.params"
2021-10-03 03:48:44,668 [INFO]  Epoch[0] Validation-cross-entropy=7.000218
2021-10-03 03:48:44,670 [INFO]  Epoch[0] Validation-screening_total_aeg-accuracy=0.000000
2021-10-03 03:48:44,700 [INFO]  Epoch[1] Batch [0-23]	Speed: 14083.32 samples/sec	cross-entropy=5.211098	screening_total_aeg-accuracy=0.000000
2021-10-03 03:48:44,728 [INFO]  Epoch[1] Train-cross-entropy=5.103684
2021-10-03 03:48:44,730 [INFO]  Epoch[1] Train-screeni

In [ ]:
imputed

,trap_id,type,successful,total_aegypti_caught,screening_total_aeg,screening_wmel_aeg,latitude,longitude,Comuna 1,Comuna 18,Comuna 20,November,December,January,February,March,April,July,screening_total_aeg_imputed
410,ASP-TYU2CO2W,Aspirator,True,1.0,NaN,NaN,3.383875,-76.559589,0,1,0,0,1,0,0,0,0,0,0.879449
799,ASP-TOJPPGIK,Aspirator,True,3.0,NaN,NaN,3.395431,-76.547335,0,1,0,0,0,0,0,0,0,1,2.992562
800,ASP-8B3Q13ST,Aspirator,True,9.0,NaN,NaN,3.394321,-76.552521,0,1,0,0,0,0,0,0,0,1,7.865554
801,ASP-4O3VJ2BP,Aspirator,True,3.0,NaN,NaN,3.394386,-76.551026,0,1,0,0,0,0,0,0,0,1,2.992562
802,ASP-Y5O47158,Aspirator,True,10.0,NaN,NaN,3.391404,-76.550628,0,1,0,0,0,0,0,0,0,1,8.570206
803,C18_PK_PR,Aspirator,True,5.0,NaN,NaN,3.391836,-76.547670,0,1,0,0,0,0,0,0,0,1,4.756177
804,ASP-9V9LWAGE,Aspirator,True,13.0,NaN,NaN,3.389942,-76.548023,0,1,0,0,0,0,0,0,0,1,10.412471
805,ASP-ZJGTMWGN,Aspirator,True,8.0,NaN,NaN,3.387627,-76.548740,0,1,0,0,0,0,0,0,0,1,7.160899
807,ASP-9INAM3UW,Aspirator,True,2.0,NaN,NaN,3.373521,-76.545673,0,1,0,0,0,0,0,0,0,1,2.109384
808,ASP-547FBSXH,Aspirator,True,1.0,NaN,NaN,3.379236,-76.546418,0,1,0,0,0,0,0,0,0,1,1.194672


In [ ]:
df_test['screening_total_aeg']=imputed['screening_total_aeg_imputed']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
df_ovitrap_imp_aeg = pd.concat([df_train, df_test])

In [ ]:
df_ovitrap_imp_aeg

,trap_id,type,successful,total_aegypti_caught,screening_total_aeg,screening_wmel_aeg,latitude,longitude,Comuna 1,Comuna 18,Comuna 20,November,December,January,February,March,April,July
1,ASP-3WQ6DML5,Aspirator,True,2.0,2.000000,2.0,3.458807,-76.557513,1,0,0,1,0,0,0,0,0,0
3,ASP-QUEZ5LU5,Aspirator,True,7.0,7.000000,5.0,3.454069,-76.562949,1,0,0,1,0,0,0,0,0,0
10,ASP-IREL246U,Aspirator,True,17.0,17.000000,6.0,3.457746,-76.583165,1,0,0,1,0,0,0,0,0,0
12,ASP-O4K94YY6,Aspirator,True,2.0,2.000000,1.0,3.455246,-76.578502,1,0,0,0,1,0,0,0,0,0
13,ASP-IREL246U,Aspirator,True,15.0,15.000000,4.0,3.457746,-76.583165,1,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
817,ASP-P29RC0I4,Aspirator,True,5.0,4.756177,NaN,3.390597,-76.553677,0,1,0,0,0,0,0,0,0,1
818,C18_PK_PR_3,Aspirator,True,8.0,7.160899,NaN,3.389345,-76.556232,0,1,0,0,0,0,0,0,0,1
819,ASP-72LFRBU2,Aspirator,True,2.0,2.109384,NaN,3.391474,-76.558260,0,1,0,0,0,0,0,0,0,1
820,ASP-BMVKZFAU,Aspirator,True,2.0,2.109384,NaN,3.385350,-76.547214,0,1,0,0,0,0,0,0,0,1


In [ ]:
#now impute positives

In [ ]:
df_train1 = df_ovitrap_imp_aeg.dropna(subset=['screening_wmel_aeg'])
df_test1 = df_ovitrap_imp_aeg[df_ovitrap_imp_aeg['screening_wmel_aeg'].isnull()]

In [ ]:
#Initialize a SimpleImputer model
imputer = datawig.SimpleImputer(
    input_columns=['screening_total_aeg','Comuna 1','Comuna 18','Comuna 20','November','December','January','February','March','April','July'], # column(s) containing information about the column we want to impute
    output_column= 'screening_wmel_aeg', # the column we'd like to impute values for
    output_path = 'imputer_model' # stores model data and metrics
    )

#Fit an imputer model on the train data
imputer.fit(train_df=df_train1, num_epochs=50)

#Impute missing values and return original dataframe with predictions
imputed = imputer.predict(df_test1)

/usr/local/lib/python3.7/dist-packages/datawig/calibration.py:92: RuntimeWarning: invalid value encountered in log
  return np.log(probas)
/usr/local/lib/python3.7/dist-packages/datawig/calibration.py:59: RuntimeWarning: invalid value encountered in greater_equal
  bin_mask = (top_probas >= bin_lower) & (top_probas < bin_upper)
/usr/local/lib/python3.7/dist-packages/datawig/calibration.py:59: RuntimeWarning: invalid value encountered in less
  bin_mask = (top_probas >= bin_lower) & (top_probas < bin_upper)


In [ ]:
imputer.explain() #TRY THIS

In [ ]:
df_test1['screening_wmel_aeg']=imputed['screening_wmel_aeg_imputed']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
df_test1

,trap_id,type,successful,total_aegypti_caught,screening_total_aeg,screening_wmel_aeg,latitude,longitude,Comuna 1,Comuna 18,Comuna 20,November,December,January,February,March,April,July
410,ASP-TYU2CO2W,Aspirator,True,1.0,0.879449,0.022329,3.383875,-76.559589,0,1,0,0,1,0,0,0,0,0
799,ASP-TOJPPGIK,Aspirator,True,3.0,2.992562,1.119249,3.395431,-76.547335,0,1,0,0,0,0,0,0,0,1
800,ASP-8B3Q13ST,Aspirator,True,9.0,7.865554,1.743273,3.394321,-76.552521,0,1,0,0,0,0,0,0,0,1
801,ASP-4O3VJ2BP,Aspirator,True,3.0,2.992562,1.119249,3.394386,-76.551026,0,1,0,0,0,0,0,0,0,1
802,ASP-Y5O47158,Aspirator,True,10.0,8.570206,1.770680,3.391404,-76.550628,0,1,0,0,0,0,0,0,0,1
803,C18_PK_PR,Aspirator,True,5.0,4.756177,1.442028,3.391836,-76.547670,0,1,0,0,0,0,0,0,0,1
804,ASP-9V9LWAGE,Aspirator,True,13.0,10.412471,1.714368,3.389942,-76.548023,0,1,0,0,0,0,0,0,0,1
805,ASP-ZJGTMWGN,Aspirator,True,8.0,7.160899,1.699536,3.387627,-76.548740,0,1,0,0,0,0,0,0,0,1
807,ASP-9INAM3UW,Aspirator,True,2.0,2.109384,0.987592,3.373521,-76.545673,0,1,0,0,0,0,0,0,0,1
808,ASP-547FBSXH,Aspirator,True,1.0,1.194672,0.824757,3.379236,-76.546418,0,1,0,0,0,0,0,0,0,1


In [ ]:
df_ovitrap_fullyimp = pd.concat([df_train1, df_test1])
df_ovitrap_fullyimp

,trap_id,type,successful,total_aegypti_caught,screening_total_aeg,screening_wmel_aeg,latitude,longitude,Comuna 1,Comuna 18,Comuna 20,November,December,January,February,March,April,July
1,ASP-3WQ6DML5,Aspirator,True,2.0,2.000000,2.000000,3.458807,-76.557513,1,0,0,1,0,0,0,0,0,0
3,ASP-QUEZ5LU5,Aspirator,True,7.0,7.000000,5.000000,3.454069,-76.562949,1,0,0,1,0,0,0,0,0,0
10,ASP-IREL246U,Aspirator,True,17.0,17.000000,6.000000,3.457746,-76.583165,1,0,0,1,0,0,0,0,0,0
12,ASP-O4K94YY6,Aspirator,True,2.0,2.000000,1.000000,3.455246,-76.578502,1,0,0,0,1,0,0,0,0,0
13,ASP-IREL246U,Aspirator,True,15.0,15.000000,4.000000,3.457746,-76.583165,1,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
817,ASP-P29RC0I4,Aspirator,True,5.0,4.756177,1.442028,3.390597,-76.553677,0,1,0,0,0,0,0,0,0,1
818,C18_PK_PR_3,Aspirator,True,8.0,7.160899,1.699536,3.389345,-76.556232,0,1,0,0,0,0,0,0,0,1
819,ASP-72LFRBU2,Aspirator,True,2.0,2.109384,0.987592,3.391474,-76.558260,0,1,0,0,0,0,0,0,0,1
820,ASP-BMVKZFAU,Aspirator,True,2.0,2.109384,0.987592,3.385350,-76.547214,0,1,0,0,0,0,0,0,0,1


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df_ovitrap_fullyimp.dtypes

trap_id                  object
type                     object
successful                 bool
total_aegypti_caught    float64
screening_total_aeg     float64
screening_wmel_aeg      float64
latitude                float64
longitude               float64
Comuna 1                  int64
Comuna 18                 int64
Comuna 20                 int64
November                  int64
December                  int64
January                   int64
February                  int64
March                     int64
April                     int64
July                      int64
dtype: object

In [ ]:
df_ovitrap_fullyimp.to_csv('df_ovitrap_fullyimp1.csv')